# Prepare global input data for MizuRoute with Döll 
Inne Vanderkelen - March 2021

#### Map reservoir parameters onto global river and catchment topology

In [5]:
# load modules
import xarray as xr
import geopandas as gpd
import pandas as pd
import netCDF4 as nc4
import numpy as np
import os 
import time


In [6]:
# load global river topo
data_dir = './data_for_mizuroute/'

ds_ntopo = xr.open_dataset(data_dir+'ntopo_hdma_mod.reorder_lake.nc')

# shapefile also containing lake_id
river_with_lake_dir = data_dir+'/river_with_lake_flag4_reorder/'
# updated HDMA topology including lakes up to 10 km²
gpd_river = gpd.read_file(data_dir+'/HDMA_hydrolakes10km_reorder/river_with_lake_flag4_10km_reorder.shp')


In [7]:
# add lake model to shapefile: 0: no lake model, 1: Döll, 2: Hanasaki
gpd_river_D03 = gpd_river
gpd_river_D03['lake_model'] = 0
gpd_river_D03['ISLAKE'] = 0
gpd_river_D03['RATECVA'] = 0
gpd_river_D03['RATECVB'] = 0


# add reservoirs as Hanasaki
#gpd_river_D03.loc[gpd_river_D03['lakeId'].isin(list(gpd_grand['lakeId'].values)),'lake_model'] = 1
#gpd_river_D03.loc[(gpd_river_D03['islake'] ==1) & (~gpd_river_D03['lakeId'].isin(list(gpd_grand['lakeId'].values))),'lake_model'] = 1
gpd_river_D03.loc[gpd_river_D03['islake'] ==1,'lake_model'] = 1
gpd_river_D03.loc[gpd_river_D03['islake'] ==1,'ISLAKE'] = 1

gpd_river_D03.loc[gpd_river_D03['islake'] ==1,'RATECVA'] = ds_ntopo['RATECVA'].max().values # coefficient
gpd_river_D03.loc[gpd_river_D03['islake'] ==1,'RATECVB'] = 1.5 # power



In [8]:
# write netcdf file with H06 parameters to merge to existing topology

# define the shapefile
shp = gpd_river_D03

# open the nc file to write
ncid = nc4.Dataset(data_dir+'HDMA_D03_parameters.nc', "w", format="NETCDF4")
# the dimension of the nc file variables is equal to the row of the shapefile
dimid_seg = ncid.createDimension('seg',len(ds_ntopo.seg))

# define the variable segId
varid = ncid.createVariable('lakeId','i8',('seg',),fill_value=-1) #assuming all the fields are ints
# Attributes
varid.long_name      = 'lakeId corresponding to HydroLAKES and GRanD'
varid.unit           = '-'
# assign the values
temp = np.array(shp['lakeId']); temp = temp.astype(int)
varid[:] = temp


# define the variable lake_model
varid = ncid.createVariable('lake_model','i8',('seg',),fill_value=-1) #assuming all the fields are ints
# Attributes
varid.long_name      = 'Lake model used (1: Döll, natural lakes; 2: Hanasaki, reservoirs)'
varid.unit           = '-'
# assign the values
temp = np.array(shp['lake_model']); temp = temp.astype(int)
varid[:] = temp


# define the variable lake_Vol
varid = ncid.createVariable('lakeVol','f8',('seg',),fill_value=-9999) #assuming all the fields are floats
# Attributes
varid.long_name      = 'Maximum lake storage'
varid.unit           = '-'
# assign the values
temp = np.array(shp['lake_Vol']) * 1000000 + 1 # 1 is to avoid division by zero
varid[:] = temp

# define the variable ISLAKE
varid = ncid.createVariable('ISLAKE','i8',('seg',),fill_value=-9999) #assuming all the fields are floats
# Attributes
varid.long_name      = ''
varid.unit           = '-'
# assign the values
temp = np.array(shp['ISLAKE'])
varid[:] = temp

# define the variable RATECVA
varid = ncid.createVariable('RATECVA','f8',('seg',),fill_value=-9999) #assuming all the fields are floats
# Attributes
varid.long_name      = 'coefficient for storage-discharge relationship power'
varid.unit           = '-'
# assign the values
temp = np.array(shp['RATECVA']) 
varid[:] = temp
  
    
# define the variable RATECVA
varid = ncid.createVariable('RATECVB','f8',('seg',),fill_value=-9999) #assuming all the fields are floats
# Attributes
varid.long_name      = 'power for storage-discharge relationship power'
varid.unit           = '-'
# assign the values
temp = np.array(shp['RATECVB']) 
varid[:] = temp    
    
ncid.Conventions = 'CF-1.6'
ncid.License     = 'The data were written by Inne Vanderkelen. They are under GPL.'
ncid.history     = 'Created ' + time.ctime(time.time())
ncid.source      = 'Written by prepare_globalinput_natlake.ipynb notebook'
ncid.close()

In [9]:
# merge Döll parameters in network topology
ds_D03_param = xr.open_dataset(data_dir + 'HDMA_D03_parameters.nc')

# correct for spaces in front of the PFAF code
#seg_pfafs = ds_ntopo['PFAF'].to_dataframe()
#seg_pfafs['PFAF'] = np.char.strip( seg_pfafs['PFAF'].values.astype(str))
#ds_ntopo['PFAF'] = seg_pfafs['PFAF']

ds_ntopo = ds_D03_param.merge(ds_ntopo, compat='override').to_netcdf(data_dir+'ntopo_hdma_mod.reorder_lake_D03.nc')

In [11]:
ds = xr.open_dataset(data_dir+'ntopo_hdma_mod.reorder_lake_D03.nc')